## Prepare count files

Install package based on [here](https://github.com/kauralasoo/rasqual/tree/master/rasqualTools)
using `devtools::install_github("kauralasoo/rasqual/rasqualTools")`. This has to be done in R console opened on a real terminal window. I installed it in conda env `testother`.

In [1]:
library(rasqualTools)
library(tibble)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
# Prepare count file
countfile <- "/project2/xinhe/ATAC-seq_10252018/CN/newCN_peaks.20_samples.counts.txt"
countm <- read.table(countfile, header=T,stringsAsFactors = F)
newheadert <- read.table("/home/simingz/run_rasqual/datarun/bamlist.txt",header=F,stringsAsFactors = F)

In [3]:
newheader <- paste0('CN_',lapply(apply(newheadert,2,strsplit,split="_")[[1]],'[[',3))

In [4]:
head(countm)

chr,start,end,peak_name,CN_02,CN_03,CN_04,CN_05,CN_06,CN_07,⋯,CN_12,CN_13,CN_14,CN_15,CN_16,CN_17,CN_18,CN_19,CN_20,CN_21
chr1,9909,10719,newCN1,583,676,637,568,643,978,⋯,751,560,948,793,546,739,1079,669,722,321
chr1,11093,11418,newCN2,51,110,57,24,60,132,⋯,63,129,98,51,52,49,93,20,97,49
chr1,19856,20555,newCN3,25,45,130,56,71,34,⋯,57,71,93,168,46,94,56,51,68,49
chr1,20674,20992,newCN4,7,26,35,16,17,16,⋯,25,22,30,44,18,21,32,18,32,22
chr1,28683,30094,newCN5,396,363,483,456,224,404,⋯,422,566,349,284,374,337,644,426,542,481
chr1,31250,31463,newCN6,16,28,36,24,31,26,⋯,16,40,27,23,33,31,31,43,23,19


In [5]:
counts_matrix <- countm[,5:dim(countm)[2]]
counts_matrix <- counts_matrix[,newheader]
rownames(counts_matrix) <-countm$peak_name
head(counts_matrix)

,CN_16,CN_08,CN_18,CN_19,CN_21,CN_10,CN_02,CN_20,CN_15,CN_14,CN_11,CN_12,CN_17,CN_06,CN_03,CN_09,CN_05,CN_04,CN_13,CN_07
newCN1,546,653,1079,669,321,986,583,722,793,948,529,751,739,643,676,673,568,637,560,978
newCN2,52,126,93,20,49,125,51,97,51,98,86,63,49,60,110,53,24,57,129,132
newCN3,46,90,56,51,49,86,25,68,168,93,76,57,94,71,45,33,56,130,71,34
newCN4,18,34,32,18,22,36,7,32,44,30,18,25,21,17,26,15,16,35,22,16
newCN5,374,515,644,426,481,475,396,542,284,349,428,422,337,224,363,344,456,483,566,404
newCN6,33,48,31,43,19,32,16,23,23,27,31,16,31,31,28,21,24,36,40,26


In [6]:
saveRasqualMatrices(list("cellTypeCN" = counts_matrix), "../../datarun/CN2", file_suffix = "atac20")

[1] "../../datarun/CN2/cellTypeCN.atac20.txt"


In [7]:
# size factor file
size_factors = rasqualCalculateSampleOffsets(counts_matrix, gc_correct = FALSE)
saveRasqualMatrices(list("cellTypeCN" = size_factors), "../../datarun/CN2", file_suffix = "size_factors")

[1] "../../datarun/CN2/cellTypeCN.size_factors.txt"


In [8]:
# count SNP number per feature and generate batch run script.
gene_data =  as_data_frame(dplyr::select(countm, peak_name,chr,start,end))
gene_data = add_column(gene_data,strand= as.integer(1),.before="start")
colnames(gene_data) <- c("gene_id", "chr", "strand","exon_starts","exon_ends")
gene_data <- gene_data %>% mutate(exon_starts = as.character(exon_starts)) %>% mutate(exon_ends = as.character(exon_ends))

In [9]:
print(gene_data[1:5,])

# A tibble: 5 x 5
  gene_id chr   strand exon_starts exon_ends
  <chr>   <chr>  <int> <chr>       <chr>    
1 newCN1  chr1       1 9909        10719    
2 newCN2  chr1       1 11093       11418    
3 newCN3  chr1       1 19856       20555    
4 newCN4  chr1       1 20674       20992    
5 newCN5  chr1       1 28683       30094    


In [10]:
gen_command <- function(chrom, gdf, vcfdir, outdir, outtag) {
    qstart <- min(as.integer(gdf$exon_starts))
    qend <- max(as.integer(gdf$exon_ends))
    tmp_file <- tempfile(pattern = paste(chrom,qstart,qend,sep="_"), tmpdir = tempdir(), fileext = ".snplist")
    system(paste0("tabix ",vcfdir,"/",chrom,".hg38withchr.AS.vcf.gz ",chrom,":",qstart,"-",qend, "| cut -f 1-3> ", tmp_file))
    snpdf <- try(read.table(tmp_file, header=F, stringsAsFactors = F))
    if(inherits(snpdf, "try-error")){
        print(paste0("No line for ",chrom, outtag,". No output .sh file."))
        return(0)
    }
    file.remove(tmp_file)
    snpdf <- as_data_frame(snpdf)
    colnames(snpdf) <- c("chr", "pos", "snp_id")
    snp_counts <- countSnpsOverlapingExons(gdf, snpdf, cis_window = 2e3)
    cmd_mainfn <- paste0(outdir,"/","mainrun_",chrom, "_", outtag)
    cmd_permfn <- paste0(outdir,"/","permrun_",chrom, "_", outtag)
    mainbashf<-file(paste0(cmd_mainfn,".sh"),"w")
    permbashf<-file(paste0(cmd_permfn,".sh"),"w")
    writeLines(paste0("rm ", cmd_mainfn, ".result.txt"), mainbashf)
    writeLines(paste0("rm ",cmd_permfn, ".permres.txt"), permbashf)
    for (rowi in 1:nrow(snp_counts)){
        featureID <- snp_counts$gene_id[rowi]
        rSNP <- snp_counts$cis_snp_count[rowi]
        fSNP <- snp_counts$feature_snp_count[rowi]
        rstart <- snp_counts$range_start[rowi]
        rend <- snp_counts$range_end[rowi]
        fstart <- snp_counts$exon_starts[rowi]
        fend <- snp_counts$exon_ends[rowi]
        cmd_main <- paste0("tabix ",vcfdir,"/",chrom,".hg38withchr.AS.vcf.gz ",chrom,":", rstart, "-",rend,
                       "|/home/simingz/run_rasqual/rasqual/bin/rasqual -y ",outdir,"/cellTypeCN.atac20.bin -k ",
                       outdir, "/cellTypeCN.size_factors.bin -n 20 -j ",featureID," -l ",rSNP," -m ",fSNP,
                       " -s ",fstart," -e ", fend, " -f ", featureID," -z -t")
        cmd_perm <- paste0(cmd_main, " -r|  cut -f 1,10-12,25") # keep lead SNP only, select column
        writeLines(paste0(cmd_main," >> ",cmd_mainfn, ".result.txt"), mainbashf)
        writeLines(paste0(cmd_perm," >> ",cmd_permfn, ".permres.txt" ), permbashf)
    }
    close(mainbashf)
    close(permbashf)
    return(snp_counts)
}

In [11]:
nlinesperjob <- 800
vcfdir <- "/home/simingz/run_rasqual/phased_data"
datarundir <- "/home/simingz/run_rasqual/datarun/CN2"

gene_data_bychr <- split(gene_data,gene_data$chr)
gene_data_bychr$chrX <- NULL
gene_data_bychr$chrY <- NULL

for (chrom in names(gene_data_bychr)){
    n <- nrow(gene_data_bychr[[chrom]])
    r <- rep(1:ceiling(n/nlinesperjob),each=nlinesperjob)[1:n]
    chromlist <- split(gene_data_bychr[[chrom]],r)
    for (tag in names(chromlist)){
        genedf <- chromlist[[tag]]
        a<-gen_command(chrom,genedf,vcfdir,datarundir, paste0("CN",tag))
    }
}

[1] "No line for chr1CN13. No output .sh file."
[1] "No line for chr13CN1. No output .sh file."


In [13]:
 a<-gen_command(chrom,genedf,vcfdir,datarundir, paste0("CN",tag))

In [16]:
head(a,n=50)

gene_id,chromosome_name,strand,exon_starts,exon_ends,range_start,range_end,feature_snp_count,cis_snp_count
newCN113054,chr9,1,137465050,137465259,137463050,137467259,2,42
newCN113055,chr9,1,137466653,137467037,137464653,137469037,11,71
newCN113056,chr9,1,137479201,137479538,137477201,137481538,4,98
newCN113057,chr9,1,137479640,137479886,137477640,137481886,11,97
newCN113058,chr9,1,137480334,137480904,137478334,137482904,8,96
newCN113059,chr9,1,137487776,137488069,137485776,137490069,6,93
newCN113060,chr9,1,137498462,137498795,137496462,137500795,8,110
newCN113061,chr9,1,137499371,137499618,137497371,137501618,7,111
newCN113062,chr9,1,137506665,137506901,137504665,137508901,7,87
newCN113063,chr9,1,137509963,137510397,137507963,137512397,12,89


In [18]:
counts_matrix["newCN503",]

,CN_16,CN_08,CN_18,CN_19,CN_21,CN_10,CN_02,CN_20,CN_15,CN_14,CN_11,CN_12,CN_17,CN_06,CN_03,CN_09,CN_05,CN_04,CN_13,CN_07
newCN503,18,23,36,8,16,30,32,20,18,28,29,21,29,8,36,22,15,11,21,34
